In [4]:
import sqlalchemy as sqla
import os
import pandas as pd
import numpy as np
import os
import copy
from copy import deepcopy
import datetime

## import local lb
# import util_dataframe

pd.set_option('display.max_columns', None)
## show all array values pandas
np.set_printoptions( threshold= np.inf )


import seaborn as sns
import matplotlib.pyplot as plt


# plt.style.use( 'classic' )
plt.style.use( 'seaborn-v0_8-white' )
# plt.style.use( 'dark_background' )
# plt.style.available

import inspect


In [5]:
### postgreSQL connection config
username = 'madhur_devkota_u'
password_ml_db = os.environ.get( 'pg__ml_db' ) 
host = 'mlmaindbro'
port = '5432' 
database = 'ml_db'

#### 

cnx_str = f'postgresql+psycopg2://{username}:{password_ml_db}@{host}:{port}/{database}'
sql_engine1 = sqla.create_engine(cnx_str)
del password_ml_db, cnx_str

In [6]:
start_dt, end_dt = '05/01/2025', '12/14/2025'
customer_id = 788

In [7]:
### read insurance0 and insurance1 batch status tables 

## batch table, one record per week, however might have multiple records for a week
_bs_ins0_df = pd.read_sql( f"""
	select *
	from payorintel.batch_status_insurance0_t bsit
	where cust_id = {customer_id} and date(last_run_date) between '{start_dt}' and '{end_dt}' 
	order by date(last_run_date) desc

""", sql_engine1 ).reset_index( drop= True )
print( _bs_ins0_df.shape ) ; _bs_ins0_df.head(3)


col_ls_0 = [ 'last_run_date', 'cust_id',
			 'temp_batch_insurance0_received', 'batch_insurance0_inserted',
			 'number_acct_selected_efx' ]

bs_ins0_df = (  copy.deepcopy( _bs_ins0_df )
	### !! rem this table most^4 probably have multiple records for a given date if multiple runs happened on same date
	### so make sure to agg on date after filtering, same for below tables
	.filter( col_ls_0 )
	## normalize time to midnight (useful for date comparisons but could lose time info, if req later)
	.assign( last_run_date = lambda df: pd.to_datetime( df['last_run_date'] ).dt.normalize() )
	.drop( columns= [ 'cust_id' ]	)  ## !! shouldn't be dropped if we are working for multiple customers
	## agg so as to avoid duplicate rows for a given date
	.groupby( 'last_run_date' ).sum().reset_index()  ## <-- this is important to not have multiple records for a same date
	.sort_values( 'last_run_date', ascending= False, ignore_index= True )
)
print( bs_ins0_df.shape ); bs_ins0_df


bs_ins1_df = deepcopy( bs_ins0_df )
bs_ins2_df = deepcopy( bs_ins0_df )


(228, 13)
(228, 4)


In [ ]:
def membr(*args):
    for e_arg in args:
        print(e_arg)
      
        
membr( 'asd', '123', 'arfdf' ) 

asd
123
arfdf


In [16]:
asd = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}
for kk, vv in asd.items():
    print(f'{kk} --> {vv}')

key1 --> value1
key2 --> value2
key3 --> value3


In [ ]:
def kwarg_func(**kwargs):
    # for e_k, e_v in kwargs.items():
    #     print(e_k, e_v)
	print(f'{kwargs} --> {type(kwargs)}')
        
kwarg_func(key1='value1', key2='value2') 
kwarg_func(a=1, b=2, c=3, d=4)

{'key1': 'value1', 'key2': 'value2'} --> <class 'dict'>
{'a': 1, 'b': 2, 'c': 3, 'd': 4} --> <class 'dict'>


In [25]:
def membr(*args):
    print(f'{args} --> {type(args)}')
    # for e_arg in args:
    #     print(e_arg)
      
        
membr( 'asd', '123', 'arfdf' ) 

('asd', '123', 'arfdf') --> <class 'tuple'>


In [ ]:
def make_summarizer(column_name):
    """
    This is the outer 'factory' function. 
    It takes a column_name and creates a specialized function for it.
    """
    def summarizer(dataframe):
        """
        This is the inner function (the closure).
        It 'remembers' the column_name from the outer function's scope.
        """
        return dataframe[column_name].sum()

    # The factory returns the specialized inner function
    return summarizer

# Create two specialized summarizer functions (closures)
sum_sales = make_summarizer('Sales')
sum_quantity = make_summarizer('Quantity')

# # Now use these specialized functions on our DataFrame
total_sales = sum_sales(df)
total_quantity = sum_quantity(df)

# print(f"The 'sum_sales' closure remembers 'Sales'. Total Sales: {total_sales}")
# print(f"The 'sum_quantity' closure remembers 'Quantity'. Total Quantity: {total_quantity}")

In [36]:
sum_sales = make_summarizer('Sales')
print(  inspect.getsource(sum_sales)  )

    def summarizer(dataframe):
        """
        This is the inner function (the closure).
        It 'remembers' the column_name from the outer function's scope.
        """
        return dataframe[column_name].sum()



In [46]:
## closure

def add_to_column(  col_name  ):
    
    def with_value(  value  ):
        
        def apply(  df  ):
            df = df.copy()
            df[ col_name ] = df[ col_name ] + value
            return df
        
        return apply
    
    return with_value


add_to_Age__add2col = add_to_column( col_name= 'Age' )
# add_to_Salary = add_to_column( col_name= 'Salary' )

add_5toAge__withValue = add_to_Age__add2col( value= 5 )


print(  inspect.getsource(add_5toAge__withValue)  )

        def apply(  df  ):
            df = df.copy()
            df[ col_name ] = df[ col_name ] + value
            return df



In [90]:
def min_row_count_rule(  min_rows  ):

    def check(  df  ):
        return {
            "passed": len( df ) >= min_rows,
            "actual_rows": len( df ),
            "expected_min": min_rows
        }

    return check

daily_rule = min_row_count_rule(  200  )
monthly_rule = min_row_count_rule(  1000  )

daily_rule(  bs_ins0_df  )
# monthly_rule(  bs_ins0_df  )



{'passed': True, 'actual_rows': 228, 'expected_min': 200}

In [ ]:
## currying example

def standardize(  stats_dictn  ):
    
    def with_column(  col_str  ):
        
        def apply(  df  ):
            df = df.copy()
            mean = stats_dictn[ col_str ][ "mean" ]
            std  = stats_dictn[ col_str ][ "std"  ]
            df[ col_str ] = ( df[ col_str ] - mean ) / std
            
            return df
        
        return apply
    
    return with_column



stats_dictn1 = {  ## from expt
    "temp_batch_insurance0_received" : { "mean": 3250,  "std": 150 },
    "batch_insurance0_inserted"   :    { "mean": 350,    "std": 50 }
}




In [ ]:
bs_ins0_df = ( deepcopy( bs_ins0_df )
	.pipe(  standardize( stats_dictn= stats_dictn1 )("temp_batch_insurance0_received")  )
	.pipe(  standardize( stats_dictn= stats_dictn1 )("batch_insurance0_inserted")  )
)
bs_ins0_df

,batch_id,last_run_date,cust_id,temp_batch_insurance0_received,batch_insurance0_inserted,insurance0_npi_inserted,number_acct_selected_efx,audit_part_id,audit_step_id,audit_process,run_date,ebureau_sent_count,demo_mart_job_number
0,373601,2025-12-14 00:00:01,788,-0.720000,-0.98,None,37,3,0,BATCH DONE,2025-12-14 04:40:22.320166,0,1.0
1,373406,2025-12-13 00:00:01,788,0.513333,0.32,None,48,3,0,BATCH DONE,2025-12-13 08:24:06.018782,0,1.0
2,373248,2025-12-12 00:00:01,788,0.073333,0.20,None,60,3,0,BATCH DONE,2025-12-12 11:30:34.562701,0,1.0
3,372928,2025-12-11 00:00:01,788,-0.893333,1.28,None,54,3,0,BATCH DONE,2025-12-11 04:57:58.992838,0,1.0
4,372697,2025-12-10 00:00:01,788,-0.653333,2.14,None,65,3,0,BATCH DONE,2025-12-10 05:17:14.626834,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,329531,2025-05-05 00:00:01,788,1.193333,-2.66,None,29,3,0,BATCH DONE,2025-05-05 06:22:37.149997,0,NaN
224,329428,2025-05-04 00:00:01,788,2.300000,-0.96,None,31,3,0,BATCH DONE,2025-05-04 07:16:59.789764,0,NaN
225,329295,2025-05-03 00:00:01,788,3.273333,0.76,None,60,3,0,BATCH DONE,2025-05-03 09:09:36.089248,0,NaN
226,329081,2025-05-02 00:00:01,788,2.353333,-0.20,None,43,3,0,BATCH DONE,2025-05-02 06:04:41.542505,0,NaN


In [ ]:
bs_ins0_df.head()

,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx
0,2025-12-14,3142,301,37
1,2025-12-13,3327,366,48
2,2025-12-12,3261,360,60
3,2025-12-11,3116,414,54
4,2025-12-10,3152,457,65


In [95]:
### partial

import functools as ftool

## function that partial uses later
def flag_highThreshold_func( df, col, threshold_val ):
	df = df.copy()
	df[ f'flagHigh__{col}' ] = df[col] > threshold_val
	return df

In [97]:
bs_ins0_df.head(2)

,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx
0,2025-12-14,3142,301,37
1,2025-12-13,3327,366,48


In [96]:
## prior to partial
df1 = flag_highThreshold_func( bs_ins0_df, 'temp_batch_insurance0_received', 3000 )

print( df1['flagHigh__temp_batch_insurance0_received'].value_counts() )

df1['flagHigh__temp_batch_insurance0_received'].head()

flagHigh__temp_batch_insurance0_received
True     217
False     11
Name: count, dtype: int64


0    True
1    True
2    True
3    True
4    True
Name: flagHigh__temp_batch_insurance0_received, dtype: bool

In [ ]:
## with partial
flag_highT_partial = ftool.partial( flag_highThreshold_func, 
                                   col= 'temp_batch_insurance0_received',
                                   threshold_val= 3000 )
print( type(flag_highT_partial), '\n', flag_highT_partial )

<class 'functools.partial'> 
 functools.partial(<function flag_highThreshold_func at 0x000001D0B97E50C0>, col='temp_batch_insurance0_received', threshold_val=3000)


In [ ]:
df2_partial = flag_highT_partial( bs_ins0_df)
print( df2_partial['flagHigh__temp_batch_insurance0_received'].value_counts() )
df2_partial.head(2)

flagHigh__temp_batch_insurance0_received
True     217
False     11
Name: count, dtype: int64


,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx,flagHigh__temp_batch_insurance0_received
0,2025-12-14,3142,301,37,True
1,2025-12-13,3327,366,48,True


In [ ]:
bs_ins0_df.groupby( 'last_run_date', as_index= False ).apply( 
	ftool.partial( flag_highThreshold_func, col= 'temp_batch_insurance0_received', threshold_val= 3000 )
)

D:\Users\madhur.devkota\AppData\Local\Temp\1\ipykernel_2652\3801185448.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bs_ins0_df.groupby( 'last_run_date', as_index= False ).apply(


,,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx,flagHigh__temp_batch_insurance0_received
0,227,2025-05-01,3539,468,56,True
1,226,2025-05-02,3603,340,43,True
2,225,2025-05-03,3741,388,60,True
3,224,2025-05-04,3595,302,31,True
4,223,2025-05-05,3429,217,29,True
...,...,...,...,...,...,...
223,4,2025-12-10,3152,457,65,True
224,3,2025-12-11,3116,414,54,True
225,2,2025-12-12,3261,360,60,True
226,1,2025-12-13,3327,366,48,True


In [114]:
## config driven pipeline

rule_ls = [
    ftool.partial( flag_highThreshold_func, col= 'temp_batch_insurance0_received', threshold_val= 3500 ),
    ftool.partial( flag_highThreshold_func, col= 'number_acct_selected_efx', threshold_val= 50 ),
]

bs_ins0_df1 = deepcopy( bs_ins0_df )
for erule in rule_ls:
     bs_ins0_df1 = erule( bs_ins0_df1 )

In [115]:
bs_ins0_df1

,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx,flagHigh__temp_batch_insurance0_received,flagHigh__number_acct_selected_efx
0,2025-12-14,3142,301,37,False,False
1,2025-12-13,3327,366,48,False,False
2,2025-12-12,3261,360,60,False,True
3,2025-12-11,3116,414,54,False,True
4,2025-12-10,3152,457,65,False,True
...,...,...,...,...,...,...
223,2025-05-05,3429,217,29,False,False
224,2025-05-04,3595,302,31,True,False
225,2025-05-03,3741,388,60,True,True
226,2025-05-02,3603,340,43,True,False


#### Classes

You are cleaning many DataFrames using:

	- the same column
	- the same threshold
	- across notebooks, scripts, pipelines

Passing the same parameters to functions over and over is noisy and error-prone.

In [ ]:
## prior class
def drop_low_values(  df, column, min_value  ):
    return df[  df[ column ] >= min_value  ]


In [ ]:
drop_low_values(  bs_ins0_df, 'number_acct_selected_efx', 50  )
drop_low_values(  bs_ins1_df, 'number_acct_selected_efx', 60  )
drop_low_values(  bs_ins2_df, 'number_acct_selected_efx', 75  )

,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx
25,2025-11-19,3332,495,75
67,2025-10-08,3417,476,75
108,2025-08-28,3106,385,76
116,2025-08-20,3137,455,81


In [ ]:
## classes

class MinValueFilter_Class:
    # Add a constructor. The constructor stores state
    def __init__( obj, col, min_value ):
        obj.col = col
        obj.min_value = min_value
        
    ## add method
    def apply_method( obj, df ):
        minRow_df = df[  df[ obj.col ] >= obj.min_value  ]
        return minRow_df
    


In [ ]:
filter_efx_obj = MinValueFilter_Class(  'number_acct_selected_efx', 72  )
# So the object 'filter_efx_obj' is a config holder (column + threshold), made from the Class, that can be applied to any DataFrame.
print( type(filter_efx_obj)  )
df1 = filter_efx_obj.apply_method( bs_ins0_df ) 
df2 = filter_efx_obj.apply_method( bs_ins1_df )  
df3 = filter_efx_obj.apply_method( bs_ins2_df )
df1

<class '__main__.MinValueFilter_Class'>


,last_run_date,temp_batch_insurance0_received,batch_insurance0_inserted,number_acct_selected_efx
5,2025-12-09,3161,480,72
25,2025-11-19,3332,495,75
40,2025-11-04,3304,488,72
67,2025-10-08,3417,476,75
108,2025-08-28,3106,385,76
116,2025-08-20,3137,455,81


In [150]:
class MinValueFilter_Class:

    def __init__(  obj, column, min_value  ):
        obj.column = column
        obj.min_value = min_value
        obj.applied_count = 0

    def class_method_1(  obj, df  ):
        obj.applied_count += 1
        return df[  df[ obj.column ] >= obj.min_value  ]


In [155]:
filter_efx_obj = MinValueFilter_Class(  'number_acct_selected_efx', 72  )

df1 = filter_efx_obj.class_method_1( bs_ins0_df ) 
df2 = filter_efx_obj.class_method_1( bs_ins1_df )  
df3 = filter_efx_obj.class_method_1( bs_ins2_df )
df3 = filter_efx_obj.class_method_1( bs_ins2_df )

In [156]:
filter_efx_obj.applied_count

4

In [10]:
### multiple dependent methods in class

class ZScoreStandardizer_Class:
    
    def __init__( obj, col ):
        obj.col = col
        obj.mean = None
        obj.std = None
        
    def calc_meanStd_method( obj, df ):
        obj.mean = df[ obj.col ].mean()
        obj.std = df[ obj.col ].std()
        return obj
    
    def standardize_method( obj, df ):
        _df = df.copy()
        _df[ obj.col ] = ( _df[ obj.col] - obj.mean) / obj.std
        return _df

In [21]:
df_train = pd.DataFrame(  { 'income': [40000, 52000, 61000, 47000] }  )
df_new   = pd.DataFrame(  { 'income': [80000, 30000, 50000] }  )
                        
df_train

scaler_obj1 = ZScoreStandardizer_Class( col= 'income' )
scaler_obj_method1 =  scaler_obj1.calc_meanStd_method( df_train ) 

print( scaler_obj_method1.mean, scaler_obj_method1.std )

scaler_obj_method1.standardize_method( df_train )




50000.0 8831.760866327846


,income
0,-1.132277
1,0.226455
2,1.245505
3,-0.339683
